<a href="https://colab.research.google.com/github/jorgemar723/3DObjectReconstruction/blob/main/notebooks/real_photos_reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>